In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import openpyxl as opl

ModuleNotFoundError: No module named 'openpyxl'

In [3]:
calibrated_home_subscribers_mobility_df = pd.read_csv('../../data/mobility_private/month_home_vs_day_location_per_day-20200424.csv')

calibrated_home_subscribers_mobility_df = calibrated_home_subscribers_mobility_df.dropna(axis=0)
calibrated_home_subscribers_mobility_df = calibrated_home_subscribers_mobility_df[calibrated_home_subscribers_mobility_df['home_region'] < 100]

calibrated_home_subscribers_mobility_df['region'] = calibrated_home_subscribers_mobility_df['region'].astype(int).astype(str).map(lambda x: f'w_{x}')
calibrated_home_subscribers_mobility_df['home_region'] = calibrated_home_subscribers_mobility_df['home_region'].astype(int).astype(str).map(lambda x: f'w_{x}')

region = sorted(calibrated_home_subscribers_mobility_df['region'].unique())
home_region = sorted(calibrated_home_subscribers_mobility_df['home_region'].unique())
days = sorted(calibrated_home_subscribers_mobility_df['day'].unique())

expanded_idx = [(
    day,
    h_reg,
    reg) for day in days for h_reg in home_region for reg in region]

calibrated_home_subscribers_mobility_df = calibrated_home_subscribers_mobility_df.set_index(
    ['day', 'home_region', 'region']).reindex(expanded_idx).fillna(0)

calibrated_home_subscribers_mobility_df = calibrated_home_subscribers_mobility_df.reset_index()

calibrated_home_subscribers_mobility_df[calibrated_home_subscribers_mobility_df['home_region'].isnull()]

,day,home_region,region,mean_duration,stdev_duration,count


# Transition rate

In [4]:
calibrated_home_subscribers_mobility_df['date'] = pd.to_datetime(calibrated_home_subscribers_mobility_df['day'])
calibrated_home_subscribers_mobility_df['weekday'] = calibrated_home_subscribers_mobility_df.date.dt.weekday

daily_population_subscribers = calibrated_home_subscribers_mobility_df.groupby(['date', 'home_region'])['count'].sum()
day_src_dst_location_count = calibrated_home_subscribers_mobility_df.groupby(['date', 'home_region', 'region'])['count'].sum()

daily_region_transition_probability = day_src_dst_location_count.divide(
    daily_population_subscribers
).groupby(level=['date', 'home_region']).cumsum()

daily_region_transition_probability = daily_region_transition_probability.reset_index()
daily_region_transition_probability['weekday'] = daily_region_transition_probability['date'].dt.weekday
daily_region_transition_probability.rename(columns={'count': 'prob'}, inplace=True)

daily_region_transition_probability = daily_region_transition_probability.groupby(['weekday', 'home_region', 'region'], sort=True)['prob'].mean()
daily_region_transition_probability = daily_region_transition_probability.unstack('region')

daily_region_transition_probability.to_csv('../../data/mobility_private/mobility_output/daily_region_transition_probability-new-district_dummy.csv')
daily_region_transition_probability

region                    w_1      w_10      w_11      w_12      w_13  \
weekday home_region                                                     
0       w_1          0.942708  0.942800  0.942810  0.942835  0.942929   
        w_10         0.000461  0.893818  0.894367  0.895160  0.918132   
        w_11         0.000249  0.002355  0.884661  0.924126  0.939445   
        w_12         0.000157  0.001378  0.016853  0.952607  0.959711   
        w_13         0.000268  0.017955  0.021105  0.024122  0.921207   
...                       ...       ...       ...       ...       ...   
6       w_6          0.000475  0.000706  0.000744  0.000876  0.001278   
        w_60         0.004767  0.005019  0.005067  0.005175  0.005448   
        w_7          0.000785  0.001016  0.001042  0.001099  0.001338   
        w_8          0.000380  0.000560  0.000591  0.000671  0.000921   
        w_9          0.000680  0.000864  0.000889  0.000961  0.001287   

region                   w_14      w_15      w_16      w_17      w_18  ...  \
weekday home_region                                                    ...   
0       w_1          0.942966  0.942972  0.942990  0.943213  0.943456  ...   
        w_10         0.926040  0.926776  0.954548  0.954815  0.963099  ...   
        w_11         0.974144  0.974383  0.974906  0.975081  0.976315  ...   
        w_12         0.960532  0.960593  0.960833  0.960952  0.962167  ...   
        w_13         0.923981  0.924242  0.925634  0.925888  0.936658  ...   
...                       ...       ...       ...       ...       ...  ...   
6       w_6          0.001404  0.001417  0.001536  0.002027  0.004052  ...   
        w_60         0.005563  0.005580  0.005667  0.006186  0.006922  ...   
        w_7          0.001438  0.001466  0.001544  0.001890  0.003147  ...   
        w_8          0.001038  0.001069  0.001172  0.001420  0.002685  ...   
        w_9          0.001394  0.001403  0.001503  0.001661  0.003662  ...   

region                   w_55      w_56      w_57      w_58      w_59  \
weekday home_region                                                     
0       w_1          0.994337  0.996497  0.997536  0.997946  0.998340   
        w_10         0.997168  0.997624  0.998179  0.998224  0.998325   
        w_11         0.998950  0.999099  0.999320  0.999351  0.999392   
        w_12         0.999008  0.999130  0.999256  0.999279  0.999350   
        w_13         0.997806  0.998180  0.998555  0.998588  0.998680   
...                       ...       ...       ...       ...       ...   
6       w_6          0.058135  0.058504  0.058788  0.058831  0.058893   
        w_60         0.060028  0.066746  0.069140  0.101300  0.111626   
        w_7          0.046025  0.046737  0.047036  0.047142  0.047255   
        w_8          0.024040  0.024318  0.024537  0.024582  0.024661   
        w_9          0.030788  0.031122  0.031278  0.031319  0.031368   

region                    w_6      w_60       w_7       w_8  w_9  
weekday home_region                                               
0       w_1          0.998537  0.999290  0.999818  0.999922  1.0  
        w_10         0.998818  0.998936  0.999690  0.999897  1.0  
        w_11         0.999469  0.999501  0.999922  0.999985  1.0  
        w_12         0.999586  0.999644  0.999889  0.999971  1.0  
        w_13         0.999104  0.999179  0.999720  0.999902  1.0  
...                       ...       ...       ...       ...  ...  
6       w_6          0.967229  0.967318  0.985331  0.996196  1.0  
        w_60         0.111927  0.998394  0.999739  0.999905  1.0  
        w_7          0.056775  0.057081  0.977585  0.998236  1.0  
        w_8          0.042586  0.042726  0.077378  0.988940  1.0  
        w_9          0.041096  0.041179  0.051730  0.073055  1.0  

[420 rows x 60 columns]

# Generate districts to lockdown for different scenarios

In [5]:
daily_region_transition_rates = day_src_dst_location_count.divide(daily_population_subscribers)

daily_region_transition_rates = daily_region_transition_rates.reset_index()

daily_region_transition_rates['weekday'] = daily_region_transition_rates['date'].dt.weekday
daily_region_transition_rates.rename(columns={'count': 'prob'}, inplace=True)

daily_region_transition_rates = daily_region_transition_rates.groupby(['weekday', 'home_region', 'region'], sort=True)['prob'].mean()
daily_region_transition_rates = daily_region_transition_rates.unstack('region')

In [6]:
daily_day_src_dst_location_count = day_src_dst_location_count.reset_index()

daily_day_src_dst_location_count['weekday'] = daily_day_src_dst_location_count['date'].dt.weekday
daily_day_src_dst_location_count.rename(columns={'count': 'prob'}, inplace=True)

daily_day_src_dst_location_count = daily_day_src_dst_location_count.groupby(['weekday', 'home_region', 'region'], sort=True)['prob'].mean()
daily_day_src_dst_location_count = daily_day_src_dst_location_count.unstack('region')

daily_day_src_dst_location_count.groupby(level=['home_region']).mean()

region,w_1,w_10,w_11,w_12,w_13,w_14,w_15,w_16,w_17,w_18,...,w_55,w_56,w_57,w_58,w_59,w_6,w_60,w_7,w_8,w_9
home_region,,,,,,,,,,,,,,,,,,,,,
w_1,254888.535714,26.867063,3.432540,9.194444,32.765873,10.829365,2.531746,7.178571,71.736111,79.664683,...,149.468254,723.101190,341.049603,141.400794,129.275794,61.827381,246.998016,162.291667,36.295635,24.029762
w_10,30.496032,51497.363095,32.998016,57.182540,1234.763889,469.551587,48.462302,1661.410714,19.410714,475.821429,...,7.442460,31.170635,37.067460,3.876984,6.825397,34.021825,8.043651,53.515873,13.174603,7.688492
w_11,3.462302,37.351190,14254.333333,630.369048,248.849206,544.246032,4.043651,8.081349,2.736111,23.226190,...,0.807540,2.970238,3.367063,0.428571,0.934524,2.075397,1.248016,6.938492,1.192460,0.601190
w_12,7.424603,59.297619,722.644841,44726.702381,336.442460,36.380952,3.789683,12.214286,6.865079,60.914683,...,1.845238,7.785714,6.916667,1.188492,3.994048,11.688492,3.257937,14.412698,4.001984,2.220238
w_13,22.640873,1517.029762,256.978175,261.430556,72214.785714,234.162698,27.946429,124.484127,25.251984,859.055556,...,12.603175,30.738095,33.015873,3.587302,7.007937,38.650794,7.769841,46.373016,16.869048,10.787698
w_14,8.926587,484.206349,660.220238,32.646825,228.865079,50296.690476,441.392857,580.591270,9.575397,85.630952,...,3.912698,12.281746,9.934524,1.700397,2.545635,9.460317,2.436508,15.420635,3.813492,2.190476
w_15,2.825397,60.438492,3.053571,3.396825,24.880952,427.126984,9182.892857,24.833333,2.253968,16.793651,...,0.236111,2.238095,2.025794,0.373016,0.686508,1.644841,0.680556,3.347222,0.728175,0.208333
w_16,5.509921,1794.753968,12.021825,16.890873,134.666667,755.204365,29.041667,26901.644841,8.380952,226.404762,...,6.394841,6.521825,9.859127,1.259921,2.367063,11.831349,2.430556,15.882937,4.827381,2.313492
w_17,40.849206,11.779762,2.771825,5.954365,16.535714,6.394841,1.126984,4.740079,31612.668651,61.390873,...,7.053571,61.095238,30.281746,4.982143,9.876984,35.408730,10.845238,49.555556,10.156746,5.226190


## Greatest number of outbound movements

In [7]:
values = daily_day_src_dst_location_count.groupby(level=['home_region']).mean().values
np.fill_diagonal(values, 0)
largest_outbound_sorted_index = values.sum(axis=1).argsort()[::-1]

In [8]:
outbound_districts = np.array(region)[largest_outbound_sorted_index]
outbound_districts

array(['w_2', 'w_18', 'w_31', 'w_1', 'w_36', 'w_7', 'w_26', 'w_28',
       'w_56', 'w_23', 'w_19', 'w_57', 'w_13', 'w_6', 'w_21', 'w_10',
       'w_30', 'w_34', 'w_3', 'w_45', 'w_17', 'w_60', 'w_33', 'w_35',
       'w_8', 'w_55', 'w_59', 'w_16', 'w_32', 'w_27', 'w_4', 'w_22',
       'w_38', 'w_25', 'w_58', 'w_14', 'w_5', 'w_53', 'w_37', 'w_52',
       'w_46', 'w_24', 'w_12', 'w_54', 'w_49', 'w_20', 'w_41', 'w_9',
       'w_48', 'w_11', 'w_47', 'w_40', 'w_29', 'w_44', 'w_43', 'w_51',
       'w_15', 'w_50', 'w_39', 'w_42'], dtype='<U4')

## Greatest number of inbound movements

In [9]:
largest_inbound_sorted_index = values.sum(axis=0).argsort()[::-1]

In [10]:
inbound_districts = np.array(region)[largest_inbound_sorted_index]
inbound_districts

array(['w_2', 'w_31', 'w_18', 'w_1', 'w_36', 'w_23', 'w_7', 'w_26',
       'w_13', 'w_28', 'w_56', 'w_19', 'w_6', 'w_21', 'w_57', 'w_10',
       'w_30', 'w_45', 'w_8', 'w_34', 'w_3', 'w_35', 'w_17', 'w_55',
       'w_60', 'w_46', 'w_27', 'w_32', 'w_4', 'w_33', 'w_59', 'w_16',
       'w_14', 'w_38', 'w_58', 'w_22', 'w_25', 'w_5', 'w_54', 'w_53',
       'w_24', 'w_52', 'w_37', 'w_49', 'w_20', 'w_12', 'w_9', 'w_41',
       'w_11', 'w_48', 'w_29', 'w_50', 'w_44', 'w_40', 'w_47', 'w_43',
       'w_15', 'w_51', 'w_42', 'w_39'], dtype='<U4')

# Combined inbound and outbound

In [11]:
inbound_outbound_df = pd.DataFrame({'outbound': outbound_districts, 'inbound': inbound_districts}).reset_index()
inbound_outbound_df.head()

,index,outbound,inbound
0,0,w_2,w_2
1,1,w_18,w_31
2,2,w_31,w_18
3,3,w_1,w_1
4,4,w_36,w_36


In [12]:
top_dist = ((inbound_outbound_df.set_index('outbound')['index'] + inbound_outbound_df.set_index('inbound')['index']) / 2).sort_values().head(10)
top_dist

w_2     0.0
w_31    1.5
w_18    1.5
w_1     3.0
w_36    4.0
w_7     5.5
w_26    6.5
w_23    7.0
w_28    8.0
w_56    9.0
Name: index, dtype: float64

In [13]:
top_dist.index

Index(['w_2', 'w_31', 'w_18', 'w_1', 'w_36', 'w_7', 'w_26', 'w_23', 'w_28',
       'w_56'],
      dtype='object')

In [14]:
sh = gpd.read_file('../../data/raw/shapefiles/new_districts/ZWE_adm2.shp')

In [15]:
inbound_outbound_df['inbound_district'] = inbound_outbound_df['inbound'].str.lstrip('w_').astype(int)
inbound_outbound_df['outbound_district'] = inbound_outbound_df['outbound'].str.lstrip('w_').astype(int)

In [16]:
inbound_outbound_df['inbound_district'] = inbound_outbound_df['inbound_district'].map(sh.set_index('ID_2')['NAME_2'])

In [17]:
inbound_outbound_df['outbound_district'] = inbound_outbound_df['outbound_district'].map(sh.set_index('ID_2')['NAME_2'])

In [18]:
inbound_outbound_df[['inbound_district', 'outbound_district']].to_excel('../../data/Most mobile districts_dummy.xlsx')

ModuleNotFoundError: No module named 'openpyxl'

# Duration

In [ ]:
mobility_df = calibrated_home_subscribers_mobility_df.copy()
mobility_df.head()

In [ ]:
mobility_df.rename(columns={'mean_duration': 'avg_duration', 'stdev_duration': 'stddev_duration'}, inplace=True)
mobility_df.head()

In [ ]:
regions = sorted(mobility_df['home_region'].unique())
full_idx = [(dow, src, dst) for dow in range(7) for src in regions for dst in regions]

weekday_mobility_duration_df = (
    mobility_df.groupby(
        ['weekday', 'home_region', 'region']
    )[['avg_duration', 'stddev_duration']].mean()
)
weekday_mobility_duration_df[['avg_duration', 'stddev_duration']] = weekday_mobility_duration_df[['avg_duration', 'stddev_duration']] / (60 * 60)

weekday_mobility_duration_df = weekday_mobility_duration_df.reindex(full_idx)
weekday_mobility_duration_df['avg_duration'] = weekday_mobility_duration_df['avg_duration'].fillna(24)
weekday_mobility_duration_df['stddev_duration'] = weekday_mobility_duration_df['stddev_duration'].fillna(weekday_mobility_duration_df['stddev_duration'].mean())

In [ ]:
weekday_mobility_duration_df.to_pickle('../../data/weekday_mobility_duration_count_df-new-district.pickle')
weekday_mobility_duration_df.head()

# Find seed districts

In [ ]:
old_new_df = pd.read_csv('../../data/district_relation.csv')

In [ ]:
old_new_df['new_dist_id'] = old_new_df['NEW_DIST_ID_2'].map(lambda x: f'w_{x}')

In [ ]:
old_new_df[old_new_df['ADMIN_NAME'] == 'Bulawayo Urban']

In [ ]:
old_new_df[old_new_df['ADMIN_NAME'] == 'Harare Urban']

In [ ]:
old_new_df[old_new_df['ADMIN_NAME'] == 'Goromonzi']

In [ ]:
old_new_df.sort_values('new_dist_id').drop_duplicates('new_dist_id').reset_index().head(50)